## Extract dataset used by a paper with OpenAlex API

Exemple of request for the paper "Exploring Soothness...":  https://api.openalex.org/works/https://doi.org/10.1007/978-3-031-16443-9_4?select=referenced_works

In [2]:
import requests
import numpy as np
from pprint import pprint
import csv

#Dictionnary of papers with names as key and DOIs as value. Loaded from papers.csv file
papers = {}
pap_reader = csv.DictReader(open('../data/papers.csv'))
for p in pap_reader:
    papers[p["name"]] = p["DOI"]


#Dictionnary with dataset names as key and openalex id as value. We associate an openalex ID because it's the value in the "referenced_works" field given by the API.
#To save a request, I use directly the OpenAlex ID otherwise you need to make a request with the ID to get the DOI or the name.
datasets_id = {}

#Dictionnary with dataset's name as key and context/segmented organ as value
datasets_context = {}

#The 2 informations are in datasets.csv file
ds_reader = csv.DictReader(open('../data/datasets.csv'))
for ds in ds_reader:
    datasets_id[ds["name"]] = ds["openalex_id"]
    datasets_context[ds["name"]] = ds["context"]

pprint(datasets_context)


{'ACDC': 'Cardiac',
 'BRATS': 'Brain',
 'I2CVB': 'Prostate',
 'LA': 'Cardiac',
 'M&Ms': 'Prostate',
 'MSCMRSeg': 'Cardiac',
 'Medical Decathlon': 'Prostate',
 'PROMISE12': 'Cardiac',
 'Synapse': 'Multi-organ'}


Now we can query the OpenAlex API with paper's DOI and check in the "referenced_works" field which contains a list of OpenAlex ID if one of them match a dataset we're working with

In [3]:
#Base url for the request
base_url = "https://api.openalex.org/works/https://doi.org/"

#To only get the field of referenced works in the response
query_param = {
    "select":"referenced_works"
}


#Will be filled with datasets found in the "referenced works" response of each paper
datasets_used = {p:[] for p in papers}

for paper_name in papers:
    request_url = base_url + papers[paper_name]
    request = requests.get(request_url, params=query_param)
    json_response = request.json()
    for ds in datasets_id:
        if datasets_id[ds] in json_response['referenced_works']:
            datasets_used[paper_name].append(ds)
pprint(datasets_used)

{'Exploring Smoothness and Class-Separation for Semi-supervised Medical Image Segmentation': ['ACDC',
                                                                                              'LA'],
 'MaxStyle: Adversarial Style Composition for Robust Medical Image Segmentation': ['PROMISE12',
                                                                                   'I2CVB'],
 'Swin-Unet: Unet-like Pure Transformer for Medical Image Segmentation': [],
 'TransUNet: Transformers Make Strong Encoders for Medical Image Segmentation': [],
 'nnFormer: Volumetric Medical Image Segmentation via a 3D Transformer': ['ACDC']}


## Extract Paper Task with the abstract contains in API Response

Exemple of request for the paper "Exploring Soothness...":  https://api.openalex.org/works/https://doi.org/10.1007/978-3-031-16443-9_4?select=abstract_inverted_index

Here we're searching for keywords in the abstract of the papers by using the same query as before but looking in the "abstract_inverted_index" field which contains the list of word in the abstract. The goal here is not to find the dataset but the task of the paper (brain segmentation, heart segmentation, etc...)

In [4]:
#Base url for the request
base_url = "https://api.openalex.org/works/https://doi.org/"

#To only get the field of referenced works in the response
query_param = {
    "select":"abstract_inverted_index"
}

key_words = ["Heart","Cardiac","Prostate","Multi-organ","Brain","Liver","Aorta","Gallblader","Spleen","Kidney","Pancreas","Stomach","Lung","Breast","Chest","Skin","Eye","Retina"]
paper_theme = {p:[] for p in papers}

for paper_name in papers:
    request_url = base_url + papers[paper_name]
    request = requests.get(request_url, params=query_param)
    json_response = request.json()
    abstract_words = list(json_response["abstract_inverted_index"])
    for kw in key_words:
        if np.array([kw.upper() in w.upper() for w in abstract_words]).any():
            paper_theme[paper_name].append(kw)

paper_theme

{'Exploring Smoothness and Class-Separation for Semi-supervised Medical Image Segmentation': [],
 'MaxStyle: Adversarial Style Composition for Robust Medical Image Segmentation': ['Cardiac',
  'Prostate'],
 'nnFormer: Volumetric Medical Image Segmentation via a 3D Transformer': [],
 'Swin-Unet: Unet-like Pure Transformer for Medical Image Segmentation': ['Cardiac',
  'Multi-organ'],
 'TransUNet: Transformers Make Strong Encoders for Medical Image Segmentation': ['Cardiac',
  'Multi-organ']}

## Merge the information between context obtained with abstract and context obtained with referenced datasets 

Finally we can combine the 2 previous informations (referenced datasets and task in the abstract) to be more accurate on the task of a paper. As we can get the context/task of a dataset in datasets.csv file.

In [5]:
print("==============================Before fusion using only abstract==============================")
pprint(paper_theme)
for paper_name in papers:
    datasets = datasets_used[paper_name]
    for ds in datasets:
        if datasets_context[ds] not in paper_theme[paper_name]:
            paper_theme[paper_name].append(datasets_context[ds])
print("\n==============================After fusion using both abstract and referenced papers==============================")
pprint(paper_theme)

==============================Before fusion using only abstract==============================
{'Exploring Smoothness and Class-Separation for Semi-supervised Medical Image Segmentation': [],
 'MaxStyle: Adversarial Style Composition for Robust Medical Image Segmentation': ['Cardiac',
                                                                                   'Prostate'],
 'Swin-Unet: Unet-like Pure Transformer for Medical Image Segmentation': ['Cardiac',
                                                                          'Multi-organ'],
 'TransUNet: Transformers Make Strong Encoders for Medical Image Segmentation': ['Cardiac',
                                                                                 'Multi-organ'],
 'nnFormer: Volumetric Medical Image Segmentation via a 3D Transformer': []}

==============================After fusion using both abstract and referenced papers==============================
{'Exploring Smoothness and Class-Separation for Semi-supervise

## Extract papers that use a dataset with OpenAlex

Exemple of request for ACDC: https://api.openalex.org/works?page=2&filter=cites:W2804047627

Gather papers that cite one of the dataset we want to monitor

### How does it proceed ?

2. For each dataset, make a get request to https://api.openalex.org/works/filter=cites:{openalexID_of_the_dataset}
3. From the API response, we iterate through the pages (a page contains a limited number of references) to get the DOI and year of the paper that cites the dataset
4. Export the result in paper_OpenAlex.csv 
Each line contains the title of the paper,the doi,the publication year, a dataset name. So we have a line per dataset a paper is referencing.

Request the API

In [10]:
#Dictionnary containing with dataset as key and a list of papers
paper_using = {ds:[] for ds in datasets_id}


for ds in datasets_id:
    #The list of paper referencing the dataset is decomposed in multiple pages so we have to iterate with the query parameter "page" to get them all.
    next_page = True
    page_number = 1
    while next_page:
        #Definition of the request
        base_url = "https://api.openalex.org/works"
        query_param = {
            "filter":f"cites:{datasets_id[ds]}",
            "page":page_number
        }
        request = requests.get(base_url,params=query_param)

        if request.status_code == 200:
            request_json = request.json()
            
            #For each paper referencing the dataset we get the title (with a little transformation to remove "," and "\n" inside of them), doi and publication year
            for res in request_json["results"]:
                title = res["title"]
                title = title.replace(",","")
                title = title.replace("\n","")
                doi = res["doi"]
                if doi is not None:
                    doi = doi[16:] #Remove the https://doi.org/
                    paper_using[ds].append((title,doi,res["publication_year"]))

            #If the results field is empty that mean we are at the last page so we can continue to the next dataset
            #otherwise we need to go to next page of the current dataset
            if not request_json["results"]:
                next_page = False
            else:
                page_number += 1
        else:
            next_page = False
        

Export the result in papers_openalex.csv

In [11]:
with open("../extracted_csv/paper_openalex.csv","w") as file:
    file.write(f"name,DOI,publication_year,dataset_used")
    for ds in datasets_id:
        lst_papers = paper_using[ds]
        for pap in lst_papers:
            file.write(f"\n{pap[0]},{pap[1]},{pap[2]},{ds}")